In [71]:
import sqlalchemy
from sqlalchemy.sql.schema import Column
from sqlalchemy import String, Integer
from sqlalchemy.orm import declarative_base, sessionmaker
import numpy as np
import requests
import pandas as pd
from tqdm._tqdm_notebook import tqdm
#!pip3 install openpyxl
from sqlalchemy.orm import sessionmaker

In [72]:
engine = sqlalchemy.create_engine('sqlite:///natorMaps.db', echo=False)
Base = declarative_base()
class Municipio(Base):
    __tablename__ = 'municipio' # campo obrigatório
    id = Column(Integer, primary_key=True) # campo obrigatório
    malha = Column(String(100000))
    malha_g = Column(String(100000))
    view_box_malha = Column(String(100))
    nome_municipio = Column(String(100))
    codigo_ibge = Column(String(100))
    cor_do_municipio = Column(String(100))
    name_funcao_click = Column(String(100))
    caixa_de_texto = Column(String(400))
    ano_de_referencia = Column(String(20))
    uf = Column(String(20))
    codigo_uf = Column(Integer)
    populacao_estimada = Column(Integer)

In [74]:
malhas = np.load('malhas_brazil.npy', allow_pickle='TRUE').all()


In [75]:
#dados = np.load("", allow_pickle=True).all()
df = pd.read_excel(
    "popBR.xls",
    sheet_name='Municípios',
    header=1,
    nrows=5570, 
    dtype = {'UF': str, 'COD. UF': str, 'COD. MUNIC': str}
)
df['cod_ibge'] = df[["COD. UF","COD. MUNIC"]].T.apply(lambda x: x[0]+x[1])

In [94]:
Session = sessionmaker(bind=engine)
session = Session()
for i,row in tqdm(df.iterrows()):
    try:
        uf,cod_uf,cod_mun,nome_municipio,populacao_estimada,codigo_ibge = row.values
        malha = malhas[codigo_ibge]
        start,end = malha.find('<g'),malha.find('</g>')
        viewbox = malha.split('viewBox="')[1].split('" stroke-linecap=')[0]
        #cria o objeto para que em seguida seja salvo no banco.
        municipio = Municipio(
            id = codigo_ibge,
            malha = malha,
            malha_g = malha[start:end+4],
            view_box_malha = viewbox,
            nome_municipio = nome_municipio,
            codigo_ibge = codigo_ibge,
            cor_do_municipio = '',
            name_funcao_click = '',
            caixa_de_texto = '',
            ano_de_referencia = 2021,
            uf = uf,
            codigo_uf = cod_uf,
            populacao_estimada = populacao_estimada,
        )
        #verifica a existencia do dado na base
        verify = session.query(Municipio).filter_by(
            id = codigo_ibge,
            codigo_ibge = codigo_ibge,
            ano_de_referencia = 2021,
            uf = uf,
            codigo_uf = cod_uf
        ).first()
        if not verify:
            print('*****',nome_municipio)
            session.add(municipio)
    except:
        print('',end='.')
    #print(municipio.__dict__)
    #break
session.commit() #Salvando o dado no banco de dados 

0it [00:00, ?it/s]

..................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [95]:
municipios = session.query(Municipio).filter_by(uf='CE')


In [37]:
#municipios.all()[0].__dict__

In [91]:
session

In [ ]:
#list(df.itertuples())

In [ ]:
#!pip install SQLAlchemy

In [ ]:
import os 
os.listdir()

In [ ]:
#whatsapp://send?phone=+14155238886&text=join%20once-shoe